In [1]:
import json
import glob
import pandas as pd
import altair as alt

In [2]:
results = []
for file in glob.glob('../share/*/runhistory.json'):
    with open(file) as f:
        data = json.load(f)
    for row in data['data']:
        if row[1][2]['__enum__'] == 'StatusType.SUCCESS':
            config_id = row[0][0]
            config = data['configs'][str(config_id)]
            row_dict = {'communications_rounds': row[0][3], 'accuracy': 1-row[1][0], 'config':str(config).replace(',', '\n')}
            row_dict.update(config)
            results.append(row_dict)     

In [3]:
results = pd.DataFrame.from_records(results)

In [4]:
results

,communications_rounds,accuracy,config,distill_epochs,fallback,lambda_fedprox,lambda_outlier,local_epochs,local_optimizer,mixture_coefficients_base,sgd_lr,adam_lr
0,5.555556,0.0974,{'distill_epochs': 6\n 'fallback': True\n 'lam...,6,True,8.359034,2.528980,18,SGD,0.989901,0.000028,NaN
1,5.555556,0.3321,{'distill_epochs': 5\n 'fallback': True\n 'lam...,5,True,0.160385,2.958106,10,SGD,0.958214,0.005155,NaN
2,5.555556,0.6069,{'distill_epochs': 11\n 'fallback': True\n 'la...,11,True,0.000002,6.170537,4,SGD,0.173499,0.001256,NaN
3,5.555556,0.6499,{'distill_epochs': 1\n 'fallback': True\n 'lam...,1,True,1.736997,5.981669,18,Adam,0.769447,NaN,0.000155
4,5.555556,0.5975,{'distill_epochs': 11\n 'fallback': True\n 'la...,11,True,2.182995,6.594445,13,Adam,0.137206,NaN,0.033146
...,...,...,...,...,...,...,...,...,...,...,...,...
408,5.555556,0.0980,{'distill_epochs': 20\n 'fallback': False\n 'l...,20,False,0.001014,1.730005,16,SGD,0.094748,0.326743,NaN
409,5.555556,0.7317,{'distill_epochs': 19\n 'fallback': False\n 'l...,19,False,0.000001,5.963778,17,Adam,0.096288,NaN,0.000101
410,5.555556,0.7380,{'distill_epochs': 18\n 'fallback': False\n 'l...,18,False,0.005602,5.026248,5,Adam,0.076504,NaN,0.000445
411,5.555556,0.7443,{'distill_epochs': 19\n 'fallback': False\n 'l...,19,False,0.000295,8.814562,8,Adam,0.025366,NaN,0.001000


# Overall performance per amount of communication rounds

In [5]:
alt.Chart(results).transform_density(
    'accuracy',
    groupby=['communications_rounds'],
    as_=['accuracy', 'density'],
    extent=[0, 1],
).mark_area().encode(
    x="accuracy:Q",
    y='density:Q',
).facet(
    column='communications_rounds'
)

alt.FacetChart(...)

## Performance broken down into the optimized parameters and communication rounds

In [6]:
boxplot_columns = ['accuracy', 'communications_rounds', 'config', 'distill_epochs', 'fallback', 'local_epochs']
scatter_columns = ['accuracy', 'communications_rounds', 'config', 'lambda_outlier', 'mixture_coefficients_base']
log_scatter_columns = ['accuracy', 'communications_rounds', 'config', 'lambda_fedprox', 'sgd_lr', 'adam_lr']

In [7]:
alt.Chart(pd.melt(results[scatter_columns], id_vars=('accuracy', 'communications_rounds', 'config'))).mark_point().encode(
    x='value',
    y='accuracy',
    color='variable',
    tooltip='config'
).facet(
    column='variable:N',
    row='communications_rounds'
).resolve_scale(x='independent')

alt.FacetChart(...)

In [8]:
alt.Chart(pd.melt(results[log_scatter_columns], id_vars=('accuracy', 'communications_rounds', 'config'))).mark_point().encode(
    x=alt.X('value', scale=alt.Scale(type='log')),
    y='accuracy',
    color='variable',
    tooltip='config'
).facet(
    column='variable:N',
    row='communications_rounds'
)

alt.FacetChart(...)

In [9]:
alt.Chart(pd.melt(results[boxplot_columns], id_vars=('accuracy', 'communications_rounds', 'config'))).mark_point().encode(
    x='value',
    y='accuracy',
    color='variable',
    tooltip='config'
).facet(
    column='variable:N',
    row='communications_rounds'
).resolve_scale(x='independent')

alt.FacetChart(...)

## Simple Global Feature Importance via a regression tree

In [10]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
acc = results.accuracy
results.local_optimizer = results.local_optimizer.astype('category').cat.codes

In [14]:
regr = DecisionTreeRegressor(max_depth=10)
X = results[['communications_rounds', 'distill_epochs',
       'fallback', 'lambda_fedprox', 'lambda_outlier', 'local_epochs',
       'local_optimizer', 'mixture_coefficients_base', 'sgd_lr', 'adam_lr']].to_numpy(na_value=0)
y = results.accuracy
regr.fit(X, y)

DecisionTreeRegressor(max_depth=10)

In [17]:
list(zip(['communications_rounds', 'distill_epochs',
       'fallback', 'lambda_fedprox', 'lambda_outlier', 'local_epochs',
       'local_optimizer', 'mixture_coefficients_base', 'sgd_lr', 'adam_lr'], regr.feature_importances_))

[('communications_rounds', 0.010591744943236498),
 ('distill_epochs', 0.01573538471109484),
 ('fallback', 0.009064041223355553),
 ('lambda_fedprox', 0.022372570897492155),
 ('lambda_outlier', 0.016381507468911746),
 ('local_epochs', 0.062074182887134656),
 ('local_optimizer', 0.0008600915977514208),
 ('mixture_coefficients_base', 0.10664110317518105),
 ('sgd_lr', 0.15999090863817633),
 ('adam_lr', 0.5962884644576658)]

In [16]:
from sklearn.metrics import *
mean_absolute_error(regr.predict(X), y)


0.019707280365231814